1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
  
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서로 LLM에 질문과 같이 전달

In [ ]:
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters
%pip install -U langchain langchain-core langchainhub
%pip install -U pinecone

In [89]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

## chunk_size = 청크 하나를 구성하는 최대 토큰 수
## chunk_overlap = 청크들이 겹치는 토큰 수
## 예를 들어, chunk_size=1500, chunk_overlap=200이면
## 각 청크는 최대 1500 토큰까지 포함할 수 있으며,
## 각 청크는 이전 청크와 200 토큰이 겹치게 됩니다.
## 이 설정은 문서의 연속성을 유지하면서도
## 청크 간의 연결성을 높이는 데 도움이 됩니다.
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1500, 
  chunk_overlap=200
  )

loader = Docx2txtLoader("./tax_with_markdown.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

In [91]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")


In [92]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
index_name = 'tax-markdown-index'
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

if index_name not in [i["name"] for i in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# ✅ 5️⃣ from_documents() 호출 시 pinecone_client 제거
database = PineconeVectorStore.from_documents(
    document_list,
    embedding=embedding,
    index_name=index_name,
)

In [93]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query = query, k=3)


In [94]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [95]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
[Identity]
- 당신은 최고의 한국 세무 전문가입니다.
- [Context]를 참고하여 사용자의 질문에 답변하세요.

[Context]
{context}

[User Question]
{input}
""")

In [96]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

# LLM과 Prompt 준비
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

# database는 PineconeVectorStore 같은 Retriever로 변환 가능해야 함
retriever = database.as_retriever(search_kwargs={"k": 3})

# Retrieval + Combine 연결
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

# ✅ invoke 시에는 질문만 넘김
result = rag_chain.invoke({"input": query})

print(result["answer"])


연봉 5천만원인 직장인의 소득세 계산은 여러 변수에 따라 다를 수 있습니다. 기본적으로 소득세는 종합소득세에 해당하며, 과세표준에 따라 초과누진세율이 적용됩니다. 

여기서는 일반적인 소득세율을 기준으로 간단히 계산해보겠습니다. 2023년 기준 기본 소득세율은 다음과 같습니다:

- 과세표준 12백만원 이하: 6%
- 12백만원 초과 ~ 46백만원 이하: 15%
- 46백만원 초과 ~ 88백만원 이하: 24%
- 88백만원 초과 ~ 150백만원 이하: 35%
- 150백만원 초과 ~ 300백만원 이하: 38%
- 300백만원 초과 ~ 500백만원 이하: 40%
- 500백만원 초과: 42%

연봉 5천만원이 모두 과세 소득으로 간주된다고 가정하고 기본 공제를 제외한 후 계산을 진행하겠습니다.

1. 과세표준 46백만원까지의 세율은 15%이며, 400만원에 대한 세율은 24%입니다.
    - 4,600만원까지의 세금: 720만원 
    - 400만원(5,000만원 - 4,600만원)에 대한 세율 24%: 96만원

이를 합하면 총 소득세는 816만원이 됩니다.

다만, 이는 단순화한 계산으로서 실제 소득세는 개인의 소득구조, 각종 공제, 세액공제 여부에 따라 달라질 수 있습니다. 연말정산 시 공제 항목들(기초 공제, 인적공제, 보험료 공제 등)을 반영할 경우, 실제 납부할 세금은 계산한 금액보다 적을 수 있습니다. 정확한 소득세 금액을 알아보려면 국세청 홈택스에서 제공하는 연말정산 계산기 등을 활용하시거나 세무 전문가와 상담하시는 것이 좋습니다.
